In [1]:
import numpy as np
import os
import pandas as pd
def detect_settled_index(t, P, window, thresh):
	t = np.asarray(t, dtype=float)
	P = np.asarray(P, dtype=float)

	# --- rest of your function unchanged --------------------
	for start in np.arange(t[0], t[-1] - window, window):
		end   = start + window
		mask  = (t >= start) & (t <= end)
		if mask.sum() < 2:
			continue
		segment   = P[mask]		  # now works: P is an ndarray
		mean_seg  = segment.mean()
		if mean_seg == 0:
			continue
		if np.abs(segment - mean_seg).max() < thresh * mean_seg:
			return np.where(t >= start)[0][0]   # first index in settled window
	return None
def mppt_repeatability(time_list, power_list, settle_thresh=0.01, min_window=30.0, settle_window=5.0):
	"""
	Compute repeatability metrics for MPPT runs.

	Parameters:
	- time_list: list of 1D numpy arrays of time stamps (seconds)
	- power_list: list of 1D numpy arrays of power readings (same shapes as time_list)
	- settle_thresh: relative std/mean threshold for detecting steady state
	- min_window: duration (s) of the steady-state window to analyze
	- settle_window: duration (s) for the rolling stability check

	Returns:
	- dict with:
	  * "RSD_Power(%)": pooled relative standard deviation (%) of mean powers
	  * "RC_MPPT": 95% repeatability coefficient (same units as power)
	  * "means": array of per-run mean powers
	  * "stds": array of per-run std deviations
	"""
	windows = []
	for idx in range(len(time_list)):
		t = time_list[idx]
		P = power_list[idx]
		# detect start of steady-state
		idx0 = detect_settled_index(t, P, settle_window, settle_thresh)
		if idx0 is None:
			# fallback: use last min_window seconds
			start_time = t[-1] - min_window
		else:
			start_time = t[idx0]
		mask = (t >= start_time) & (t < start_time + min_window)
		windows.append(P[mask])

	means = np.array([w.mean() for w in windows])
	stds  = np.array([w.std(ddof=1) for w in windows])

	# platform metrics
	platform_rsd = stds.std(ddof=1) / means.mean()
	rc95		 = 2.77 * stds.std(ddof=1)

	return {
		"RSD_Power": platform_rsd * 100,
		"RepeatabilityCoeff_MPPT": rc95,
		"windows": windows,
		"means": means,
		"stds": stds
	}


In [2]:
def load_files(files):
	time_all = []
	power_all = []

	for file in files:
		arr = np.loadtxt(file, delimiter=",", dtype=str)
		header_row = np.where(arr == "Time")[0][0]

		meta_data = {}
		for data in arr[:header_row, :2]:
			meta_data[data[0]] = data[1]

		headers = arr[header_row, :]
		arr = arr[header_row + 1 :, :]

		header_dict = {value: index for index, value in enumerate(headers)}
		time = np.array(arr[:, header_dict["Time"]]).astype("float")

		pixel_V = arr[:, 1::2][:, 0:8].astype(float)
		pixel_mA = arr[:, 2::2][:, 0:8].astype(float)
		# print(pixel_V.shape, pixel_mA.shape)

		power = (pixel_V*pixel_mA).T
		time_all.append(time)
		power_all.append(power)

	time_collated  = [[] for _ in range(8)]
	power_collated = [[] for _ in range(8)]
	for i in range(len(time_all)):
		# print(files[i])
		for j in range(8):
			time_collated[j].append(time_all[i])
			power_collated[j].append(power_all[i][j])

	return time_collated, power_collated



In [3]:
good_cells_RSD = []
good_cells_std = []

In [ ]:

folder_path = rf"C:\Users\achen\Dropbox\code\Stability-Setup\data\Apr-30-2025 23_17_20 MPPT repeat"
folder = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

files = [f for f in folder if ("ID1" in f and "mppt" in f and "compressed" not in f)]

time_all, power_all = load_files(files)

good_cell_idx = [x - 1 for x in [1,2,3,4,5,6,7,8]]
for idx in good_cell_idx:
	results = mppt_repeatability(time_all[idx], power_all[idx])
	print(f"RSD_Power(%): {results['RSD_Power']}, RepeatabilityCoeff_MPPT: {results['RepeatabilityCoeff_MPPT']}")
	good_cells_RSD.append(results['RSD_Power'])
	good_cells_std.append(results['stds'])

RSD_Power(%): 1.231713029390777, RepeatabilityCoeff_MPPT: 0.07585190978410536
RSD_Power(%): 0.9936694693885846, RepeatabilityCoeff_MPPT: 0.046040439729029374
RSD_Power(%): 1.9744649835115786, RepeatabilityCoeff_MPPT: 0.00010082671764802579
RSD_Power(%): 0.37525081985271175, RepeatabilityCoeff_MPPT: 0.019360614215699595
RSD_Power(%): 0.5956268420275339, RepeatabilityCoeff_MPPT: 0.03738218561879932
RSD_Power(%): 0.1713809089182657, RepeatabilityCoeff_MPPT: 0.011492677367515117
RSD_Power(%): 1.5720339818249849, RepeatabilityCoeff_MPPT: 0.0972545819123626
RSD_Power(%): 0.21269807556923898, RepeatabilityCoeff_MPPT: 0.014043295139227233


In [4]:
folder_path = rf"C:\Users\achen\Dropbox\code\Stability-Setup\data\Apr-30-2025 23_17_20 MPPT repeat"
folder = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

files = [f for f in folder if ("ID2" in f and "mppt" in f and "compressed" not in f)]

time_all, power_all = load_files(files)

good_cell_idx = [x - 1 for x in [1,2,5,6,7,8]]
for idx in good_cell_idx:
	results = mppt_repeatability(time_all[idx], power_all[idx])
	print(f"RSD_Power(%): {results['RSD_Power']}, RepeatabilityCoeff_MPPT: {results['RepeatabilityCoeff_MPPT']}")
	good_cells_RSD.append(results['RSD_Power'])
	good_cells_std.append(results['stds'])


RSD_Power(%): 0.3079958190271592, RepeatabilityCoeff_MPPT: 0.018361083135744156
RSD_Power(%): 0.4949265782517376, RepeatabilityCoeff_MPPT: 0.02870707714623788
RSD_Power(%): 0.08871252388706266, RepeatabilityCoeff_MPPT: 0.0030073117290785583
RSD_Power(%): 0.343836355884467, RepeatabilityCoeff_MPPT: 0.021197109507703114
RSD_Power(%): 0.06279214104911239, RepeatabilityCoeff_MPPT: 0.004018583963090676
RSD_Power(%): 0.7161358928865493, RepeatabilityCoeff_MPPT: 0.04060241731704219


In [5]:
platform_sd_Pmax   = np.array(good_cells_std).mean()		  # repeatability (σ_r) in Pmax units
platform_rsd_Pmax  = np.array(good_cells_RSD).mean()		# repeatability as % of mean Pmax
repeat_coeff_Pmax  = 2.77 * platform_sd_Pmax	   # 95 % repeatability coeff

print(f"Platform repeatability σ_r (mW): {platform_sd_Pmax:.4f}")
print(f"Platform RSD power (%): {platform_rsd_Pmax:.2f}%")
print(f"95% repeatability coefficient (mW): {repeat_coeff_Pmax:.4f}")

Platform repeatability σ_r (mW): 0.0331
Platform RSD power (%): 0.34%
95% repeatability coefficient (mW): 0.0918
